In [1]:
# project imports
import sys
sys.path.append('/home/mark/Documents/code/drone/sumo/utils')
from runSumo import runSumo
from sumo_loop import sumo_loop

# general imports
import os
import subprocess
import numpy as np
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import pickle
import pandas as pd

# plotting params
from IPython.display import Math
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["axes.spines.right"] = False
mpl.rcParams["axes.spines.top"] = False
mpl.rcParams["lines.linewidth"] = 2

Load in the latency that we already have for two-lane roads


In [2]:
in_dir = '/home/mark/Documents/code/drone/sumo/results/latencies/3_car.p'
Data = pickle.load(open(in_dir,'rb'))
l = Data['l']/60

ratios = np.logspace(np.log10(0.001),np.log10(0.5),100)
flows = np.arange(10,5000,10) 

In [3]:
# We only want to use a subset of ratios
stop_index = 74
ratios = ratios[0:74]


In [4]:
data = []

for r_idx in range(len(ratios)):
    for f_idx in range(l.shape[1]):
        if l[r_idx,f_idx]<0.1:
            pass
        else:
            x = flows[f_idx]*ratios[r_idx]
            y = flows[f_idx]
            z = l[r_idx,f_idx]
            data.append([x,y,z])


data = np.asarray(data)

In [5]:
print(data.shape)

(29526, 3)


In [6]:
# regular grid covering the domain of the data
X,Y = np.meshgrid(np.arange(10,1510,10), flows)
XX = X.flatten()
YY = Y.flatten()


# best-fit linear plane
A_lin = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
C_lin,_,_,_ = scipy.linalg.lstsq(A_lin, data[:,2])    # coefficients

# evaluate it on grid
Z_lin = np.dot(np.c_[XX, YY, np.ones(XX.shape)], C_lin).reshape(X.shape)





In [7]:
print(C_lin)

[2.76745188e-03 1.01666496e-04 2.44332199e+00]


In [8]:
%matplotlib qt
# plot points and fitted surface
fig = plt.figure(figsize=(12,4))
ax2 = fig.add_subplot(1,3,2,projection='3d')

ax2.set_xlim3d(0,300)
ax2.set_ylim3d(0,3000)
ax2.set_zlim3d(0,250/60)

# next we plot the linear fit
ax2.set_title('Best Linear Fit')
ax2.plot_surface(X, Y, Z_lin, rstride=1, cstride=1, alpha=0.2)
ax2.scatter(data[:,0], data[:,1], data[:,2], c='r', s=0.1)

plt.show()